In [ ]:
# data.Pipeline
# data.Example
# data.Iterator.splits
# data.field
# data.dataSet
# data.vocab.stio

1 Field对象：预处理某个字段

In [ ]:
from torchtext.legacy import data
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
# step1:构建Field对象
tokenize=lambda x:x.split()
# fix_length指定了每条文本的长度，截断补长
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=200)
LABEL=data.Field(sequential=False,use_vocab=False)

In [ ]:
# step1-1 :读取数据
train_data = pd.read_csv('../data/train_one_label.csv')
valid_data = pd.read_csv('../data/valid_one_label.csv')
test_data = pd.read_csv("../data/test.csv")

In [ ]:
# step2:构建Examples对象
"""
torchtext预置的Dataset类的API如下，我们必须至少传入
examples和fields这两个参数。examples为由torchtext中的Example对象构造的列表，Example为对数据集中一条数据的抽象。fields可简单理解为每一列数据和Field对象的绑定关系，在下面的代码中将分别用train_examples和test_examples来构建训练集和测试集的examples对象，train_fields和test_fields数据集的fields对象
"""
# get_dataset构造并返回Dataset所需的examples和fields
def get_dataset(csv_data, text_field, label_field, test=False):
	# id数据对训练在训练过程中没用，使用None指定其对应的field
    # comment_text Fileld,toxic为的 name
    #  comment_text需要对应到表头的属性值吗，还是随便取一个就好？
    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("comment_text", text_field), ("toxic", label_field)]       
    examples = []
    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['comment_text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['comment_text'], csv_data['toxic'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields

In [ ]:
# step 3:得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
valid_examples, valid_fields = get_dataset(valid_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(test_data, TEXT, None, test=True)


In [ ]:
# step4:构建Dataset数据集:class torchtext.data.Dataset(examples, fields, filter_pred=None)
train = data.Dataset(train_examples, train_fields)
valid = data.Dataset(train_examples, train_fields)
test = data.Dataset(train_examples, train_fields)

In [ ]:
TEXT.build_vocab(train)
print(len(train))
vocab=TEXT.vocab
len(vocab)

In [ ]:
# step5 ：构建迭代器
train_iter, val_iter, test_iter = data.Iterator.splits(
        (train, valid, test), sort_key=lambda x: len(x.comment_text_2),
        batch_sizes=(10, 10, 10), device=-1)
batch = next(iter(train_iter))
print("batch text: ", batch.comment_text) # 对应 Fileld 的 name
print("batch label: ", batch.toxic)

In [ ]:
from collections import OrderedDict
def convert_list2dict(convert_list):
    list_dict = OrderedDict()
    for index, word in enumerate(convert_list):
        list_dict[word] = index
    return list_dict

In [ ]:
import tqdm,torch
# 词汇表的词典: token->index
text_field_words_dict=TEXT.vocab.itos
#这样不是读的所有的词的数量？
word_count = len(text_field_words_dict)
print(word_count)
path="../word2vec/glove.sentiment.conj.pretrained.txt"
embedding_dim=300
embeddings = np.zeros((int(word_count), int(embedding_dim)))
def get_pretained_embedding(path,text_field_words_dict):
    #如果没有词汇表，自建一个词汇表
    inword_list={}
    if not isinstance(text_field_words_dict, dict):
        text_field_words_dict = convert_list2dict(text_field_words_dict)
    with open(path,encoding="utf-8") as f:
        lines = f.readlines()
        lines = tqdm.tqdm(lines)
        for line in lines:
            line = line.strip().split(" ")
            index = text_field_words_dict.get(line[0])  # digit or None
            vector = np.array([i for i in line[1:]], dtype='float32')
            embeddings[index] = vector
            inword_list[index] = 1
    f.close()
    OOVWords = word_count - len(inword_list)
    print("All Words = {}, InWords = {}, OOVWords = {}".format(word_count, len(inword_list), OOVWords, oov_radio))
    return torch.from_numpy(embeddings).float()
get_pretained_embedding(path,text_field_words_dict)

In [ ]:
word_count = len(text_field_words_dict)
print('The number of wordsDict is {} \nThe dim of pretrained embedding is {}\n'.format(str(word_count),
                                                                                           str(embedding_dim)))
embeddings = np.zeros((int(word_count), int(embedding_dim)))

inword_list = {}
with open(path, encoding='utf-8') as f:
    lines = f.readlines()
    lines = tqdm.tqdm(lines)
    for line in lines:
        lines.set_description("Processing")
        #只剩数字
        values = line.strip().split(" ")
        if len(values) == 1 or len(values) == 2:
            continue
        # value[0]是token 词典可以将token转为index 
        index = text_field_words_dict.get(values[0])  # digit or None
        if index:
            vector = np.array([float(i) for i in values[1:]], dtype='float32')
            embeddings[index] = vector
            inword_list[index] = 1
    f.close()
print("oov words initial by avg embedding, maybe take a while......")
sum_col = np.sum(embeddings, axis=0) / len(inword_list)     # avg
for i in range(len(text_field_words_dict)):
    if i not in inword_list and i != padID:
        embeddings[i] = sum_col

OOVWords = word_count - len(inword_list)
oov_radio = np.round(OOVWords / word_count, 6)
print("All Words = {}, InWords = {}, OOVWords = {}, OOV Radio={}".format(
word_count, len(inword_list), OOVWords, oov_radio))

In [ ]:
# step 8: 使用 Pytorch 的 Embedding Layer 来解决 embedding lookup 问题。
embed = Embedding(len(vocab), emb_dim)
embed.weight.data.copy_(vocab.vectors)